# Fun with Flags

This notebook shows you how to interpret SDSS-V targeting flags that are represented as a single column.

Andy Casey (andrew.casey@monash.edu)
updated: Dec 19, 2024 - Sean Morrison

You will need the `sdss_semaphore` package (version ``0.2.5``) to easily interpret the SDSS flags. You can install it with:

```pip install sdss-semaphore==0.2.5```


In [1]:
import numpy as np
import os
from astropy.io import fits
from sdss_semaphore.targeting import TargetingFlags

# Let's load the Astra allVisits file.
# At the time of writing (2023-10-11) this contains targeting flags for BOSS spectra (v6_1_1) and APOGEE DR17 (no APOGEE SDSS-V).
image = fits.open(os.path.expandvars("$MWM_ASTRA/ipl-3/summary/mwmAllVisit-0.5.0.fits"))

In [2]:
# Let's look at the header information.
image[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
                                                                                
        Metadata                                                                
                                                                                
V_ASTRA = '0.5.0   '           / Astra version                                  
CREATED = '23-11-16 00:00:06'  / File creation time (UTC %y-%m-%d %H:%M:%S)     
                                                                                
        HDU Descriptions                                                        
                                                                                
COMMENT HDU 0: Summary infor

In [3]:
# Let's play with flags for spectra observed by BOSS.
try:
    SDSSC2BV = image[1].header.get('SDSSC2BV',default=1)
except:
    SDSSC2BV = 1
flags = TargetingFlags(image[1].data["SDSS5_TARGET_FLAGS"], sdssc2bv = SDSSC2BV)

In [4]:
# Let's count the number of targets assigned to each carton (and skip empty cartons)
for label, count in flags.count(skip_empty=True).items():
    print(f"{label}: {count}")

ops_std_boss_tic_0.1.3: 2024
ops_std_eboss_0.5.0: 32272
ops_std_boss_0.5.0: 92657
ops_std_boss_red_0.5.0: 11631
ops_std_apogee_0.5.0: 4828
mwm_cb_gaiagalex_apogee_0.5.0: 119
mwm_cb_gaiagalex_boss_0.5.0: 30285
mwm_cb_cvcandidates_boss_0.5.0: 550
mwm_galactic_core_0.5.0: 3884
mwm_cb_uvex3_0.5.0: 496
mwm_cb_uvex4_0.5.0: 830
mwm_snc_100pc_boss_0.5.0: 23890
mwm_snc_250pc_apogee_0.5.0: 1713
mwm_snc_250pc_boss_0.5.0: 577
mwm_yso_disk_apogee_0.5.0: 5129
mwm_yso_disk_boss_0.5.0: 8593
mwm_yso_embedded_apogee_0.5.0: 19
mwm_yso_nebula_apogee_0.5.0: 1
mwm_yso_variable_apogee_0.5.0: 9234
mwm_yso_variable_boss_0.5.0: 10134
mwm_yso_cmz_apogee_0.5.0: 179
mwm_yso_cluster_apogee_0.5.0: 7786
mwm_yso_cluster_boss_0.5.0: 13657
mwm_rv_short_fps_0.5.0: 6
mwm_legacy_ir2opt_0.5.0: 18914
mwm_dust_core_0.5.0: 28
mwm_wd_core_0.5.0: 23739
mwm_tess_planet_0.5.0: 3964
mwm_snc_100pc_apogee_0.5.0: 1632
mwm_cb_uvex1_0.5.0: 9034
mwm_cb_uvex2_0.5.0: 22686
mwm_tessrgb_core_0.5.0: 3005
ops_std_boss_tic_0.5.0: 44242
mwm_rv_l

# Flags have attributes

The counts above are for targets assigned to a specific carton, with a specific version. Sometimes we don't care about the carton version and we just want to count by the effective number of targets assigned to that carton (over all versions), or we might want to count by the number of targets assigned to each mapper, or some other attribute.BufferError

The `TargetingFlags.mapping` attribute contains a reference dictionary for all cartons, including their attributes. Here's how we can count targets by those attributes:


In [5]:
for mapper, count in flags.count_by_attribute("mapper").items():
    print(f"{mapper}: {count:,}")

mwm: 340,583
ops: 281,127
bhm: 20,601
open: 244,699


In [6]:
for program, count in flags.count_by_attribute("program").items():
    print(f"{program}: {count:,}")

mwm_snc: 47,421
mwm_cb: 51,501
mwm_halo: 21,973
mwm_yso: 56,822
mwm_rv: 766
mwm_ob: 98,649
ops_std: 234,624
mwm_wd: 26,603
mwm_gg: 0
mwm_planet: 6,115
bhm_aqmes: 588
bhm_filler: 5,489
mwm_tessrgb: 3,849
bhm_csc: 17,770
ops_sky: 0
bhm_rm: 1,528
bhm_spiders: 1,929
mwm_dust: 47
mwm_legacy: 18,978
mwm_galactic: 4,697
mwm_filler: 28,594
mwm_tess_ob: 0
mwm_erosita: 4,490
open_fiber: 244,699
commissioning: 54,365
SKY: 0
mwm_tessob: 0
mwm_magcloud: 0
mwm_bin: 3,488
mwm_validation: 1,809
mwm_monitor: 0


# I own a carton. Where are my spectra?

You can efficiently construct boolean masks to select spectra that are assigned to a carton, program, mapper, or any other attribute.

The easiest way to do this is with the following methods:
- `TargetingFlags.in_carton_pk`: If you know the database primary key (`targetdb.carton.pk`) of your carton
- `TargetingFlags.in_carton_name`: If you know the name of your carton.
- `TargetingFlags.in_carton_label`: If you know the name and version (name_ver) of your carton
- `TargetingFlags.in_mapper`: To select by mapper (e.g., BHM, MWM)
- `TargetingFlags.in_program`: To select by program name.
- `TargetingFlags.in_alt_name`: To select by alternative name, curated with thanks by Marina Koukel.
- `TargetingFlags.in_alt_program`: To select by alternative program name, curated with thanks by Marina Koukel.

If you're not sure what your carton's alternative name or program is, you might find these properties useful:

- `TargetingFlags.all_mappers`: a tuple of all unique mapper names;
- `TargetingFlags.all_programs`: a tuple of all unique program names;
- `TargetingFlags.all_carton_names`: a tuple of all unique carton names;
- `TargetingFlags.all_alt_carton_names`: a tuple of all unique alternative names;
- `TargetingFlags.all_alt_programs`: a tuple of all unique alternative program names;

or you can look this up from `TargetingFlags.mapping`.

In [9]:
print("Mappers:")
for mapper in flags.all_mappers:
    print(f"\t{mapper}")
    
print("Programs")
for program in flags.all_programs:
    print(f"\t{program}")

print("Cartons names")
for name in flags.all_carton_names:
    print(f"\t{name}")

print("Alternative cartons names")
for alt_name in flags.all_alt_carton_names:
    print(f"\t{alt_name}")

print("Alternative programs")
for alt_program in flags.all_alt_programs:
    print(f"\t{alt_program}")


Mappers:
	mwm
	open
	bhm
	ops
Programs
	mwm_erosita
	mwm_snc
	mwm_dust
	bhm_filler
	mwm_tess_ob
	mwm_ob
	mwm_monitor
	ops_sky
	bhm_csc
	ops_std
	mwm_bin
	mwm_yso
	mwm_galactic
	bhm_spiders
	open_fiber
	bhm_aqmes
	mwm_magcloud
	mwm_gg
	mwm_tessrgb
	mwm_legacy
	mwm_validation
	mwm_cb
	mwm_wd
	mwm_tessob
	mwm_rv
	commissioning
	bhm_rm
	SKY
	mwm_halo
	mwm_planet
	mwm_filler
Cartons names
	manual_bright_targets_g13
	mwm_halo_sm
	manual_mwm_halo_mp_wise_apogee_single
	mwm_halo_nmp_xp_apogee
	manual_bright_target_offsets_1_g13
	mwm_snc_ext_main_apogee
	mwm_halo_mp_xp_apogee
	mwm_tess_planet
	openfibertargets_nov2020_15
	mwm_galactic_core_dist_apogee
	mwm_snc_100pc_boss
	mwm_yso_embedded_apogee_single
	mwm_erosita_compact_boss_shallow
	mwm_cb_gaiagalex_boss
	manual_bright_targets_g13_offset_fixed_7
	manual_mwm_halo_mp_bbb
	bhm_spiders_agn_sep
	mwm_monitor_m15_apogee_long
	bhm_aqmes_bonus_core
	bhm_csc_boss-dark
	mwm_ob_cepheids
	mwm_halo_vmp_xp_apogee_single
	manual_bhm_spiders_comm_lco
	bhm_a

## Select by Mapper

In [10]:
# Select things assigned to MWM
mwm_mask = flags.in_mapper("mwm")
mwm_mask

array([False, False, False, ...,  True,  True, False])

In [11]:
# The `mwm_mask` is a N-length boolean array, where N is the number of targets in the allStar file.
assert len(mwm_mask) == len(image[1].data)

In [12]:
print(f"There are {np.sum(flags.in_mapper('mwm')):,} things assigned to MWM")

There are 340,583 things assigned to MWM


## Select by Program

In [13]:
flags.in_program("mwm_wd")

array([False, False, False, ..., False, False, False])

## Select by name

In [14]:
flags.in_carton_name("mwm_snc_250pc")

array([False, False, False, ..., False, False, False])

# I have a target, how was it targeted?

If you have a CATALOGID or SDSS_ID and would like to know the targeting cartons for that source


The easiest way to do this is with the following methods:

- `TargetingFlags.get_carton_pk`: returns a tuple of lists of carton_pk database primary key (`targetdb.carton.pk`), with 1 list per target
- `TargetingFlags.get_carton_name`: returns a tuple of lists of carton names, with 1 list per target
- `TargetingFlags.get_carton_label`: returns a tuple of lists of carton labels (name+version), with 1 list per target
- `TargetingFlags.get_mapper`: returns a tuple of lists of mapper, with 1 list per target
- `TargetingFlags.get_program`: returns a tuple of lists of programs, with 1 list per target
- `TargetingFlags.get_alt_name`: returns a tuple of lists of alternative carton names, with 1 list per target (curated with thanks by Marina Koukel)
- `TargetingFlags.get_alt_program`: returns a tuple of list of alternative program names, with 1 list per target  (curated with thanks by Marina Koukel)

They all have the same syntax, with an optional index element of type int, list of int, numpy array of ints

----
If you have a CATALOGID or SDSS_ID and would like to know the targeting cartons for that source


In [15]:
catalogid = 27021598754184543
idx = np.where((image[1].data["CATALOGID21"] == catalogid) |
               (image[1].data["CATALOGID25"] == catalogid) | 
               (image[1].data["CATALOGID31"] == catalogid))[0][0]
print(f'index: {idx}')
flags.get_carton_label(index=idx)

index: 91676


(['mwm_yso_cluster_boss_0.5.0',
  'mwm_yso_cluster_apogee_0.5.17',
  'mwm_yso_cluster_boss_0.5.17',
  'mwm_yso_cluster_apogee_1.0.6',
  'mwm_yso_variable_apogee_1.0.6',
  'mwm_yso_variable_boss_single_1.0.33',
  'mwm_yso_cluster_boss_single_1.0.33'],)

In [16]:
sdssid = 87406300
idx = np.where((image[1].data["SDSS_ID"] == sdssid))[0][0]
print(f'index: {idx}')
flags.get_carton_label(index=idx)

index: 91676


(['mwm_yso_cluster_boss_0.5.0',
  'mwm_yso_cluster_apogee_0.5.17',
  'mwm_yso_cluster_boss_0.5.17',
  'mwm_yso_cluster_apogee_1.0.6',
  'mwm_yso_variable_apogee_1.0.6',
  'mwm_yso_variable_boss_single_1.0.33',
  'mwm_yso_cluster_boss_single_1.0.33'],)

---
Similarly if you have a list of SDSS_IDs, you can supply a list or array or indexes

In [17]:
sdssids = [87508291, 87508257]
idx = np.where(np.isin(image[1].data["SDSS_ID"], sdssids))[0]
print(f'index: {idx}')
flags.get_carton_label(index=idx)

index: [91772 91773]


(['ops_std_boss_ps1dr2_0.5.0', 'ops_std_boss_ps1dr2_1.0.29'],
 ['mwm_wd_core_0.5.0', 'mwm_wd_pwd_boss_1.0.42'])

---

Finally, you supply the default `None`  value to get the source property for all the rows 

**Note if there a large number of rows, it will take a while to compute**

In [ ]:
mini_data = image[1].data[[1,2,3]]
flag1 = TargetingFlags(mini_data["SDSS5_TARGET_FLAGS"])
flag1.get_carton_label()